# Used Cars EDA
### COMP 4447
### Ainsley McCutcheon & Ted Guevel

## Dataset and Motivation:

This analysis will explore many used cars across multiple websites to see if we can determine the best rating for selecting a used car. We would like to determine if specific used car websites have better inventory, have overlapping inventory, and/or have better prices when compared to other websites. We will scrape various used car websites to collect a number of features assocaited with their inventory of used cars, use feature engineering to process difficult and missing data, and graph trends and correlations of the data. 

## Task Definition and Research Question:

Are used car webistes biased towards certain Make, Mileage, and Prices of used cars? 
Can we generate a rating that we can consider "best" when buying a used car? 

We will input html code from used car websites and output our findings and generated rating for the collected car data. 

## Literature Review:

We used resources from lectures from this class and the data visualization course to build our methodology on tackling this project. We belive our work is novel compared to the listings and ratings on specific used car websites because it offers an unbiased inventory (not website specific) and has an unbiased rating (not number of views specific or paid by owner). 

## Data Scraping:

In [1]:
#Import required packages/libraries:
from bs4 import BeautifulSoup as bsoup
from selenium import webdriver
import requests
import pandas as pd
import re
import ast
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import emoji

Below is are the urls for the various car website pages that have used cars in Denver (80225 zip code).

### Cars.com data scraping, parsing, and dataframe creation:
We will first parse the data from Cars.com and build a dataframe.

In [2]:
#100 Cars per page in Denver zip code 80225 (cars.com):
url_1 = "https://www.cars.com/shopping/results/?list_price_max=&makes[]=&maximum_distance=30&models[]=&page=1&page_size=100&stock_type=used&zip=80225"

In [3]:
#verify response:
response = requests.get(url_1)
print(response.status_code)

200


In [4]:
#generate bsoup:
html_soup = bsoup(response.text, 'html.parser')

In [5]:
#Response for Cars.com:
content_list = html_soup.find_all('div', attrs={'class': 'vehicle-card'})
#Used to sift through and find the div, span, h, etc. tags and names associated with our variables

print(content_list)

[<div class="vehicle-card inventory-result" data-listing-id="b4b3119c-6ad3-4c94-91dd-6f80d43b49cb" data-override-payload='{"sponsored_type":"premier","vertical_position":1}' data-tracking-id="1" data-tracking-type="srp-vehicle-card" id="vehicle-card-b4b3119c-6ad3-4c94-91dd-6f80d43b49cb-sponsored" phx-hook="VehicleCard" phx-target="6">
<a class="image-gallery-link vehicle-card-visited-tracking-link" href="/vehicledetail/b4b3119c-6ad3-4c94-91dd-6f80d43b49cb/" rel="nofollow" tabindex="-1"></a>
<div class="vehicle-card-main js-gallery-click-card">
<div class="vehicle-card-photos js-gallery-click-gallery">
<cars-filmstrip totalcount="33">
<div class="gallery-wrap">
<div aria-label="Photo 1 of 33 of 2020 Chevrolet Suburban LT" class="image-wrap" data-index="0">
<img alt="Photo 1 of 33 of 2020 Chevrolet Suburban LT" class="vehicle-image" loading="lazy" src="https://platform.cstatic-images.com/large/in/v2/7aa60755-c731-5a41-a1f8-ddd21269eb53/67db5c49-e29b-48a1-9514-30285a2d7826/UAWX8CrT5KCVe_5

In [6]:
#Extract Name, Mileage, Price, and VIN from Cars.com Data:

names = []
for item in content_list:
    if item.find('h2') != None:
        names.append(item.find('h2').getText())
    else:
        names.append(0)
#print(names)

miles = []
for item in content_list:
    if item.find('div', attrs={'class': "mileage"}) != None:
        miles.append(item.find('div', attrs={'class': "mileage"}).getText())
    else:
        miles.append(0)
#print(miles)

price = []
for item in content_list:
    if item.find('span', attrs={'class': "primary-price"}) != None:
        price.append(item.find('span', attrs={'class': "primary-price"}).getText())
    else:
        price.append(0)
#print(price)

vin = []
    
for item in content_list:
    if item.find('a', attrs={'class':'sds-button--secondary contact-by-phone'}) != None:
        vin.append(re.search(r'data-vin="(\S+)" ',str(item))[1])
    else:
        vin.append(None)


In [7]:
#Combine Name, Mileage, and Prices into Dataframe:
data_cars = pd.DataFrame({"Name" : names, "Mileage" : miles, "Price" : price, "VIN" : vin})[["Name", "Mileage", "Price", "VIN"]]
data_cars

,Name,Mileage,Price,VIN
0,2020 Chevrolet Suburban LT,"69,872 mi.","$39,387",1GNSKHKC1LR157892
1,2017 INFINITI QX30 Premium,"78,318 mi.","$21,640",SJKCH5CR2HA018760
2,2021 BMW M440 i xDrive,"6,061 mi.","$59,992",WBA13AR03MCF46705
3,2021 Land Rover Range Rover P525 HSE Westminster,"33,909 mi.","$99,999",SALGS2SE7MA429187
4,2014 Jaguar F-TYPE V6 S,"62,172 mi.","$36,798",SAJWA6FC5E8K00264
...,...,...,...,...
96,2019 Mercedes-Benz AMG CLS 53 S-Model 4MATIC,"27,510 mi.","$68,598",WDD2J6BB7KA031739
97,2021 Jeep Grand Cherokee L Limited,"11,284 mi.","$45,999",1C4RJKBG2M8146370
98,2020 Lincoln Continental Reserve,"19,245 mi.","$49,348",1LN6L9NP4L5605606
99,2008 Toyota Avalon XLS,"135,350 mi.","$10,200",4T1BK36B08U253977


After building the dataframe, we convert the data for Mileage and Price into the correct datatype, and add a column to identify which website the data came from.

In [8]:
#Convert Mileage to number:
def mileage(row):
    return int(row.split(" mi.")[0].replace(',',''))

data_cars['Mileage'] = data_cars['Mileage'].apply(mileage)

#data
#data.dtypes

In [9]:
#Convert Price to Number:
def price(row):
    if (row == 'Not Priced') | (row == 0):
        return 0
    else:
        return int(row.split("$")[1].replace(',', ''))

data_cars['Price'] = data_cars['Price'].apply(price)

#print(data)
#data.dtypes

In [10]:
#Add website column
data_cars['website'] = 'Cars.com'
data_cars

,Name,Mileage,Price,VIN,website
0,2020 Chevrolet Suburban LT,69872,39387,1GNSKHKC1LR157892,Cars.com
1,2017 INFINITI QX30 Premium,78318,21640,SJKCH5CR2HA018760,Cars.com
2,2021 BMW M440 i xDrive,6061,59992,WBA13AR03MCF46705,Cars.com
3,2021 Land Rover Range Rover P525 HSE Westminster,33909,99999,SALGS2SE7MA429187,Cars.com
4,2014 Jaguar F-TYPE V6 S,62172,36798,SAJWA6FC5E8K00264,Cars.com
...,...,...,...,...,...
96,2019 Mercedes-Benz AMG CLS 53 S-Model 4MATIC,27510,68598,WDD2J6BB7KA031739,Cars.com
97,2021 Jeep Grand Cherokee L Limited,11284,45999,1C4RJKBG2M8146370,Cars.com
98,2020 Lincoln Continental Reserve,19245,49348,1LN6L9NP4L5605606,Cars.com
99,2008 Toyota Avalon XLS,135350,10200,4T1BK36B08U253977,Cars.com


### Autotrader.com data scraping, parsing, and dataframe creation:

Similarly to the data from Cars.com, we parse and build a dataframe from data from Autotrader.com

In [11]:
#100 Cars per page in Denver zip code 80225 (Autotrader.com):
url_2 = "https://www.autotrader.com/cars-for-sale/all-cars/denver-co-80225?dma=&searchRadius=50&isNewSearch=false&marketExtension=include&showAccelerateBanner=false&sortBy=relevance&numRecords=100"

In [12]:
#verify response
response_autotrader = requests.get(url_2)
print(response_autotrader.status_code)

200


In [13]:
#create bsoup text
html_soup_autotrader = bsoup(response_autotrader.text, 'html.parser')

In [14]:
#Response for Autotrader.com:
content_list_autotrader = html_soup_autotrader.find_all('script', attrs={'data-cmp': 'lstgSchema'})
#print(content_list_autotrader[1])

In [15]:
print(content_list_autotrader[1])

<script data-cmp="lstgSchema" type="application/ld+json">{"@context":"http://schema.org/","@type":["Product","Car"],"vehicleIdentificationNumber":"5XYRKDLF6MG063855","name":"Certified 2021 Kia Sorento SX","mpn":"Certified 2021 Kia Sorento SX","image":"https://images.autotrader.com/hn/c/59d62eabc9064b1da2d7a38952d7a473.jpg","offers":{"@type":"Offer","priceCurrency":"USD","price":42281,"priceValidUntil":"2023-10-04","itemCondition":"http://schema.org/CertifiedCondition","availability":"http://schema.org/InStock","url":"https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=659706393","seller":{"@context":"http://schema.org/","@type":"AutoDealer","name":"Fowler Kia of Longmont","telephone":"7207136612","image":"https://assets.cai-media-management.com/dealership/bf48dcbb-8f5c-40ab-8ae3-dce92f377948.jpg","address":{"@context":"http://schema.org/","@type":"PostalAddress","addressLocality":"","addressRegion":"","postalCode":"","streetAddress":""}}},"brand":{"@context":"http:/

In [16]:
#Extract Name, Mileage, Price, and VIN from Autotrader.com Data:
#Data for Mileage and Price is already converted into integers in this step.

names = []
for item in content_list_autotrader:
    s = str(item)
    names.append(' '.join(s.split('name":"')[1].split('",')[0].split(' ')[1:]))
#print(names)

miles = []
for item in content_list_autotrader:
    s = str(item)
    s = s.split('value":"')[1].split('"}')[0].replace(',', '')
    if s == '':
        miles.append(0)
    else:
        miles.append(int(s))
#print(miles)

price = []
for item in content_list_autotrader:
    p = str(item)
    p = p.split('price":')[1].split(',')[0]
    if p =='':
        price.append(0)
    else:
        price.append(int(p))
#print(price)

vin = []
for item in content_list_autotrader:
    vin.append(re.search(r'\"vehicleIdentificationNumber\":\"(\S+)\",',str(item))[1])

In [17]:
#Combine Name, Mileage, and Prices into Dataframe:
data_autotrader = pd.DataFrame({"Name" : names, "Mileage" : miles, "Price" : price, "VIN" : vin})[["Name", "Mileage", "Price", "VIN"]]
data_autotrader['website'] = 'Autotrader.com'
data_autotrader

,Name,Mileage,Price,VIN,website
0,2020 Tesla Model Y AWD,23955,55999,5YJYGDEE1LF054875,Autotrader.com
1,2021 Kia Sorento SX,11915,42281,5XYRKDLF6MG063855,Autotrader.com
2,2019 Volkswagen Jetta S w/ Driver Assistance P...,14073,19997,3VWC57BU0KM272410,Autotrader.com
3,2018 Honda Civic Si,25550,26975,2HGFC1E51JH708669,Autotrader.com
4,2019 Nissan Leaf S Plus,29309,27591,1N4BZ1CP3KC318604,Autotrader.com
...,...,...,...,...,...
98,2019 INFINITI QX50 Luxe w/ Navigation Package,29718,31198,3PCAJ5M35KF135997,Autotrader.com
99,2012 Toyota Prius Four,66067,15762,JTDKN3DUXC1496142,Autotrader.com
100,2020 Kia Soul GT-Line,9321,21353,KNDJ63AU1L7016323,Autotrader.com
101,2019 Kia Optima S,23115,22281,5XXGT4L36KG285985,Autotrader.com


### Carmax data pulling, data scraping, parsing, and dataframe creation:

Parsing and building a dataframe from the data on Carmax.com
Because Carmax.com uses dynamic html for it's search pages, we also had to use Selenium as well as BeautifulSoup.

In [ ]:
#Pulling the html from carmax.com

url_3 = "https://www.carmax.com/cars/all?location=denver+co+80121"
response_3 = webdriver.Chrome()
response_3.get(url_3)

html_soup_3 = bsoup(response_3.page_source, 'html.parser')

content_list_3 = html_soup_3.find_all('article', attrs={'class': 'car-tile'})
print(content_list_3)

In [ ]:
#Building lists of the Make/Model/Year, Mileage, Price, and VIN from the html

names_3 = []
for item in content_list_3:
    names_3.append(item.find('span', attrs={'class': "year-make"}).text + ' ' 
                 + item.find('span', attrs={'class': "model-trim"}).text)
#print(names_3)

miles_3 = []
for item in content_list_3:
    miles_3.append(item.find('span', attrs={'class': "miles"}).text.replace('K mi',',000'))
#print(miles_3)

price_3 = []
for item in content_list_3:
    price_3.append(item.find('span', attrs={'class': "price"}).text.replace('*',''))
    
vin_3 = []
vin_html_list_carmax = html_soup_3.find_all('div', attrs={'class': "kmx-typography--body-1 vehicle-identifying-info"})
for item in vin_html_list_carmax:
    vin_3.append(re.search(r'<span>VIN # (\S+)</span>',str(item))[1])

In [ ]:
#combining the data into a dataframe
data_carmax = pd.DataFrame({"Name" : names_3, "Mileage" : miles_3, "Price" : price_3, "VIN" : vin_3})[["Name", "Mileage", "Price", "VIN"]]

In [ ]:
#Convert Mileage and Price into integers
data_carmax.Mileage = data_carmax.Mileage.str.replace(',','').astype(int)

data_carmax.Price = data_carmax.Price.str.replace(',','')
data_carmax.Price = data_carmax.Price.str.replace('$','')

In [ ]:
#adding webpage column
data_carmax['website'] = 'Carmax.com'
data_carmax.head(5)

### Join the Cars.com, Autotrader.com, and Carmax.com dataframes:

Now we will join the data from all three websites. We will also do some feature engineering on the price column, check for duplicate VINs, and drop any extremely expensive cars. 

In [ ]:
#Join website data:
data = pd.concat([data_cars, data_autotrader, data_carmax])
data

In [ ]:
#verifying datatypes
data.dtypes

Since we are scraping multiple websites, we want to check whether or not we have duplicates in our dataset. Looking for duplicate VINs is the obvious solution here. 

In [ ]:
#Check for duplicates:
duplicates = data[data['VIN'].duplicated()]
duplicates

We find it very surprising that there are not more duplicates in our dataset (average <5 each time we run the code). This must mean that each dealer and individual that is trying to sell used cars is only selecting one used car website to sell through. Maybe there are high fees associated with listing on any given website, or contract requirements with the websites?

We will now remove all duplicates (if any), and drop rows that have no Make/Model/Year information for the car.

In [ ]:
#removing duplicate car entries
data.drop_duplicates(subset=['VIN'], inplace = True)

#removing rows with no Make and Model
data.dropna(subset=['Name'], inplace = True)

We now split the Year, Make, and Model out of the Name column, creating a column for each (Year, Make, and Model respectively), and remove the Name column.
To get make sure the Make and Model split correctly (no strings that should be in the Make of the car in the Model or viceversa) we created a list of car Makes to check the Name column against.

In [ ]:
#Pull Year and Make out of Name:

#Import list of popular makes from online:
makes = open("Car_Manufacturers.txt").read()

makes_str = '(' + makes.replace('\n','|') + ')'
data['Make'] = data['Name'].str.extract(r''+makes_str, re.IGNORECASE)

#Pull out year from name column:
year_regex = re.compile('^\d.[0-9]*', re.DOTALL)

def year(row):
    if re.findall(year_regex, row)==[]:
        return None
    else:
        return re.findall(year_regex, row)[0]

data['Year'] = data['Name'].apply(year)

def Model(df):
    if pd.isnull(df['Make']) != True:
        one = df['Name'].replace(df['Make'], '')
        if df['Year'] != None:
            return one.replace(df['Year'], '')

data['Model'] = data.apply(Model, axis=1)

data

In [ ]:
#Checking the data for NaN values
data.isna().sum()

In [ ]:
#Cleaning up the datatypes in the data dataframe
data.Year = data.Year.astype(int)
data.Price = data.Price.astype(int)
data.dtypes

Sometimes the websites don't list prices, and we set those to "0". Let's check for all price = 0 rows and impute the price based on the average price for that make. 

In [ ]:
#Check to see if we have any 0 values for price (not always true):
(data['Price'] == 0).sum()

In [ ]:
#Impute price based on average price for that make:

#Get Price mean for each Make:
mean = data.groupby('Make').mean('Price')


#Fill the 0 Price value with the mean price for it's respective Make:
def price_imputer(row):
    if row['Price']==0:
        return mean['Price'][row['Make']]
    else:
        return row['Price']

#Apply Funciton to df:
data['Price'] = data.apply(price_imputer, axis=1)
 
data


Finally, we will drop all the cars with prices over $250,000. And we will drop any remaing null values to remove outliers from the data.

In [ ]:
#Drop extremely high end vehicles:
data = data[data['Price']<=250000]

#Drop remaining null values (if any):
data = data.dropna(axis=0)

data.isna().sum()

Now we will make our own rating function as a funciton of average make price, year, and mileage.

We used the following function: 
$$(\frac{Avg\ Price\ of\ Make - Price}{Avg\ Price\ of\ Make})^\frac{Year}{Mileage + 1}$$

In [ ]:
#Generate a Rating Function:

#Compute average price for each make:
avg_make_price = data.Price.groupby(data.Make).mean()
avg_make_price_dict = avg_make_price.to_dict()

#Define Rating and assign it to Rating column:
# 1 added to mileage as quick way to deal with cars that have no Mileage
data['Rating'] = (data['Make'].map(avg_make_price_dict)-data['Price'])/data['Make'].map(avg_make_price_dict)**((data['Year'].astype(int)/(data['Mileage']+1)))


We transformed the results from the Rating into a scale from 0 to 5 in a Rating_Clean feature

In [ ]:
old_max = data.Rating.max()
old_min = data.Rating.min()
old_range = old_max - old_min
new_range = 5 - 0

def new_rating(row):
    return ((row - old_min)*new_range)/old_range

data['Rating_Clean'] = data['Rating'].apply(new_rating)

We used star emojis to convert the new rating into a 1 to 5 star rating

In [ ]:
#Add emoji Stars to bucket ratings 5 separate groups:
bin_title = [' ', emoji.emojize(':star:'),emoji.emojize(':star::star:'),emoji.emojize(':star::star::star:'),
             emoji.emojize(':star::star::star::star:'),emoji.emojize(':star::star::star::star::star:')]

data['Star_Rating'] = pd.cut(data['Rating_Clean'], [0,0.5,1.5,2.5,3.5,4.5,5.0],labels=bin_title)
data

In [ ]:
#Reset index, since we have overlapping index values when we joined the dataframes:
data.reset_index(drop=True, inplace=True)
data

### Data Visualization and Comparison:

In [ ]:
#Assess a pairplot of the numerical features:
sns.pairplot(data,x_vars=['Mileage', 'Price', 'Year', 'Rating_Clean'], y_vars=['Mileage', 'Price', 'Year', 'Rating_Clean'])

The Pairplot above shows some interesting trends. Our rating has a distribution that is skewed towards 0. This is becuse the newer cars with signifantally less milage then the rest tend to be high outliers in our rating function. Some obvious trends deal with Price. As price increases, mileage decreases, and as price increases, year increases. 

In [ ]:
fig = plt.figure(figsize=(17, 10))

gs = fig.add_gridspec(4, 5)
ax0 = plt.subplot(gs[:2, :2])
ax1 = plt.subplot(gs[2:, :2])
ax2 = plt.subplot(gs[:4, 2:])

#Mileage v. Price:
sns.scatterplot(data['Mileage'], data['Price'], hue=data['Year'], ax=ax0)
              
#Plot Make v. Rating:
sns.scatterplot(data['Make'], data['Rating_Clean'], hue=data['Mileage'], ax=ax1)
plt.draw()
ax1.set_title('Make v. Rating')
ax1.set_xlabel('Make')
ax1.set_ylabel('Rating_Clean')
ax1.set_xticks(ax1.get_xticks(), ax1.get_xticklabels(), rotation=90)

#Sort data baed on Make:
result = data.groupby(["Make"])['Price'].aggregate(np.median).reset_index().sort_values('Price', ascending=False)

#Plot bar graph of price v. make:
sns.barplot(x='Make', y="Price", data=data, order=result['Make'], ax=ax2)
ax2.set_title('Make v. Price')
ax2.set_xlabel('Make')
ax2.set_ylabel('Price')
ax2.set_xticks(ax2.get_xticks(), ax2.get_xticklabels(), rotation=90)

#Show Plots:
fig.suptitle('Used Cars Data', fontsize=18)
plt.tight_layout()
plt.show()

In the Rating v. Make plot above, we can see that some Makes tend to have an average higher rating, and that all Makes have lower ratings for cars with more mileage. The Price v. Make bar plot show which car brands are more high-end and which brands are more affordable. The Mileage v. Price scatter plot shows that, generally, the price of a car tends to decrease as its mileage increases. 

In [ ]:
# histplot() is axes-level function
fig, ax = plt.subplots(figsize=(15, 10))
hist = sns.kdeplot(data=data, x='Price', hue='Make', multiple='stack', ax=ax)

# we can employ any of the pyplot axes methods on our Seaborn axes-level plots.
hist.set_title('Price Distribution by Make', fontsize=12)
plt.xticks(rotation=45)
hist.spines.left.set_visible(False)
hist.spines.bottom.set_color('green')
fig.suptitle('Used Cars', fontsize=14)
ax.set_xlim(0,150000)
plt.show()

The kde plot above gives us detailed information about each make, and what price range cars they tend to make. We can see that Mercedies Benz only makes higher end cars above 30,000 (and they keep their value since we are looking at used cars). Then Makes such as Volkswagen and Ram tend to only have used cars in the mid range of 15,000 - 35,000. Finally Makes such as Ford and BMW tend to offer used cars across the entire price spectrum. 

Now we will compare the specific websites we pulled from to see if any trends develop.

In [ ]:
#Plot bar graph of Price v Website:
fig, ax = plt.subplots(figsize=(15,5))
sns.barplot(x='website', y="Price", data=data)
ax.set_title('Price v. Website')
ax.set_xlabel('Website')
ax.set_ylabel('Price')
plt.show()

Looking at the plot of Price v. Website above we can clearly see that there is a descrepancy between sites. Carmax.com deals with a lot more lower end cars than Cars.com (highest end). The price differential between each website is about a $10,000 difference. 

In [ ]:
#Plot bar graph of Rating v Website:
fig, ax = plt.subplots(figsize=(15,5))
sns.barplot(x='website', y="Rating_Clean", data=data)
ax.set_title('Rating v. Website')
ax.set_xlabel('Rating')
ax.set_ylabel('Price')
plt.show()

Even though Cars.com offers higher priced cars, Carmax.com offers a much higher rating. This is likely due to the nature of our rating function, and the fact that we don't always consider a high price to be the best value. 
*Note that the results of the plot could vary each time the code is run. 

Finally, we will look at a correlation heatmap to visualize the relationships between each of the numerical features in our web-scraped data. 

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(data[['Rating_Clean', 'Year', 'Price', 'Mileage']].corr(), annot=True, ax=ax)
plt.show()

Looking at the heatmap above, we can tell that Year and Mileage are very inversely correlated. This makes sense because cars tend to be driven year over year. Also, there is slight correlation between Price and Year, because most cars tend to loose value the older they get (but not always).

Also, the rating correlation to Year, Price, and Mileage is esentially as we would expect, since we did generate the rating based on these features. 

### Selecting the Best Cars Available:

In [ ]:
#Best high end car (>$50,000)
high_end=data[data['Price']>=50000].copy()
data.iloc[high_end['Rating_Clean'].idxmax()]

In [ ]:
#Best medium range car ($20,000 to $50000)
high_end=data[(data['Price']>=20000)&(data['Price']<50000)].copy()
data.iloc[high_end['Rating_Clean'].idxmax()]

In [ ]:
#Best low end car (<$20,000)
high_end=data[data['Price']<20000].copy()
data.iloc[high_end['Rating_Clean'].idxmax()]

Above are three different recommendations for the best rated used cars on the market out of the three websites we pulled data from. We have offered one selection for each price range. 

### Final Thoughts:



It was rewarding to generate an analysis that could scrape multiple websites for useful information when assesing where and how to purchase a used car. We found that many websites inventory does not overlap across multiple websites, so it is useful to browse numerous sites to find that perfect deal. Also, different websites tend to cater towards different price ranges of used vehicles. This project also allowed us to compare the different makes of vehicles. Some makes offer vehicles in all price ranges, while others tend to only offer cars in specific price ranges. 

Given more time, we would have liked to incorporate additional features to improve the overall accuracy or performance of our rating funciton. The websites we pulled from, and many other used car websites, have imbedded HTML code that makes it difficult to pull all of the features associated with a used car. Once we gain more pracitce with b4 syntax, regex, and HTML code, we will be able to update this analysis to incorporate more features. 